In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

In [2]:
dataset_transformator = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./datasets', train=True, transform=dataset_transformator, download=True)

In [4]:
test_dataset = torchvision.datasets.MNIST(root='./datasets', train=False, transform=dataset_transformator)

In [5]:
batch_size = 200

In [6]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class NumNet(nn.Module):
    def __init__(self):
        super(NumNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 64, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return nn.functional.log_softmax(out, dim=0)

In [8]:
net = NumNet()

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

## Тренировка

In [10]:
for epoch in range(20):
    for i, (images, labels) in enumerate(train_loader):
        outputs = net(images)
        loss = loss_func(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        label_count = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct_count = (predicted == labels).sum().item()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/20], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}, Accuracy: {correct_count / label_count * 100:.2f}%')

Epoch [1/20], Step [100/300], Loss: 0.1744, Accuracy: 95.00%
Epoch [1/20], Step [200/300], Loss: 0.1379, Accuracy: 96.50%
Epoch [1/20], Step [300/300], Loss: 0.0601, Accuracy: 97.50%
Epoch [2/20], Step [100/300], Loss: 0.1218, Accuracy: 96.50%
Epoch [2/20], Step [200/300], Loss: 0.0975, Accuracy: 97.00%
Epoch [2/20], Step [300/300], Loss: 0.0930, Accuracy: 97.00%
Epoch [3/20], Step [100/300], Loss: 0.0343, Accuracy: 99.50%
Epoch [3/20], Step [200/300], Loss: 0.0659, Accuracy: 97.50%
Epoch [3/20], Step [300/300], Loss: 0.0915, Accuracy: 96.50%
Epoch [4/20], Step [100/300], Loss: 0.0416, Accuracy: 97.50%
Epoch [4/20], Step [200/300], Loss: 0.1150, Accuracy: 97.50%
Epoch [4/20], Step [300/300], Loss: 0.0378, Accuracy: 98.00%
Epoch [5/20], Step [100/300], Loss: 0.0420, Accuracy: 99.00%
Epoch [5/20], Step [200/300], Loss: 0.0173, Accuracy: 99.00%
Epoch [5/20], Step [300/300], Loss: 0.0557, Accuracy: 98.00%
Epoch [6/20], Step [100/300], Loss: 0.0575, Accuracy: 98.00%
Epoch [6/20], Step [200/

## Тестирование

In [11]:
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy for 10000 images: {correct / total * 100} %')

Accuracy for 10000 images: 99.3 %
